In [2]:
import cv2
import numpy as np
import math
frame_org = cv2.imread('images/hsvcalibimagemarchingants2.jpg')
bot_stream_width, bot_stream_height_org = frame_org.shape[1], frame_org.shape[0]
width, height = bot_stream_width, bot_stream_height_org
height_lt = bot_stream_height_org
cv2.imshow('image', frame_org)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@0.024] global loadsave.cpp:244 findDecoder imread_('images/hsvcalibimagemarchingants2.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [5]:
# #~ Trapezium-ish mask 
# height_lt = 480; width_lt = 640; centre_x_lt = 640 // 2
# crop_bot_h = 60
# higher_crop_triangle_h = 72
# higher_crop_triangle_w = 75
# higher_crop_triangle_gap_w = 35

# mask_trapeziums = np.zeros([height_lt, width_lt], dtype="uint8")
# left_triangle_pts = np.array([[0, height_lt - crop_bot_h], [0, height_lt - higher_crop_triangle_h], [centre_x_lt - higher_crop_triangle_gap_w - higher_crop_triangle_w, height_lt - higher_crop_triangle_h], [centre_x_lt - higher_crop_triangle_gap_w , height_lt - crop_bot_h]])
# right_triangle_pts = np.array([[width_lt, height_lt - crop_bot_h], [width_lt, height_lt - higher_crop_triangle_h], [centre_x_lt + higher_crop_triangle_gap_w + higher_crop_triangle_w, height_lt - higher_crop_triangle_h], [centre_x_lt + higher_crop_triangle_gap_w , height_lt - crop_bot_h]])
# bottom_rectangle_pts = np.array([[0, height_lt], [0, height_lt - crop_bot_h], [width_lt, height_lt - crop_bot_h], [width_lt, height_lt]])

width_lt = 640 // 4
height_lt = 480 // 4
mask_trapeziums = np.zeros([480 // 4, 640 // 4], dtype="uint8")
centre_x_lt = 160//2
crop_bot_h = 62//2
higher_crop_triangle_h = 72//2
higher_crop_triangle_w = 75//2
higher_crop_triangle_gap_w = 35//2

left_triangle_pts = np.array([[0, height_lt - crop_bot_h], [0, height_lt - higher_crop_triangle_h], [centre_x_lt - higher_crop_triangle_gap_w - higher_crop_triangle_w, height_lt - higher_crop_triangle_h], [centre_x_lt - higher_crop_triangle_gap_w , height_lt - crop_bot_h]])
right_triangle_pts = np.array([[width_lt, height_lt - crop_bot_h], [width_lt, height_lt - higher_crop_triangle_h], [centre_x_lt + higher_crop_triangle_gap_w + higher_crop_triangle_w, height_lt - higher_crop_triangle_h], [centre_x_lt + higher_crop_triangle_gap_w , height_lt - crop_bot_h]])
bottom_rectangle_pts = np.array([[0, height_lt], [0, height_lt - crop_bot_h], [width_lt, height_lt - crop_bot_h], [width_lt, height_lt]])
cv2.fillPoly(mask_trapeziums, [left_triangle_pts, right_triangle_pts, bottom_rectangle_pts], 255)
mask_trapeziums = cv2.bitwise_not(mask_trapeziums)

cv2.imshow('mask', mask_trapeziums)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [12]:
# height_lt = 640
# bot_stream_width = 480
peak_triangle_gap = 0
peak_triangle_width = 60
triangle_margin = 20
mask_gap = np.zeros([height_lt, bot_stream_width], dtype="uint8")
points = np.array([[triangle_margin, height_lt-100], [bot_stream_width-triangle_margin, height_lt-100], [int(bot_stream_width/2+peak_triangle_width), int(height_lt/2)], [int(bot_stream_width/2-peak_triangle_width),int(height_lt/2)]])
cv2.fillConvexPoly(mask_gap, points, 255)
# mask_gap[:height/2]

mask_cols = np.amax(mask_gap, axis=0)
mask_line_indices_x = np.where(mask_cols==255)
print(mask_line_indices_x)
mask_line_start_x = mask_line_indices_x[0][0] if len(mask_line_indices_x[0]) else 0
mask_line_end_x = mask_line_indices_x[0][-1] if len(mask_line_indices_x[0]) else 0
max_mask_width = mask_line_end_x - mask_line_start_x

print("max width of line: ", max_mask_width)
print("max no. of pixels: ", np.sum(mask_gap) / 255)
cv2.imshow('lol', mask_gap)
cv2.waitKey(0)
cv2.destroyAllWindows()


(array([ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 2

In [9]:
x_com = np.tile(np.linspace(-1., 1., width), (height, 1))
y_com = np.array([[i] * width for i in np.linspace(1., 0, height)])
print(y_com)

[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.99791232 0.99791232 0.99791232 ... 0.99791232 0.99791232 0.99791232]
 [0.99582463 0.99582463 0.99582463 ... 0.99582463 0.99582463 0.99582463]
 ...
 [0.00417537 0.00417537 0.00417537 ... 0.00417537 0.00417537 0.00417537]
 [0.00208768 0.00208768 0.00208768 ... 0.00208768 0.00208768 0.00208768]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
#mask_gap
# peak_triangle_gap = 0
# peak_triangle_width = 60
# triangle_margin = 20
# mask_gap = np.zeros([height, width], dtype="uint8")
# points = np.array([[triangle_margin, 0], [width-triangle_margin, 0], [int(width/2+peak_triangle_width), int(height-peak_triangle_gap)], [int(width/2-peak_triangle_width), int(height-peak_triangle_gap)]])
# cv2.fillConvexPoly(mask_gap, points, 255)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [11]:

#~ Club values
# u_black = 110 #102 prev
# l_blue = np.array([96, 170, 80], np.uint8)
# u_blue = np.array([106, 245, 191], np.uint8)
# l_green = np.array([30, 50, 60], np.uint8)
# u_green = np.array([85, 255, 255], np.uint8)
# l_blue = np.array([96, 170, 80], np.uint8)
# u_blue = np.array([109, 245, 191], np.uint8)

# frame_hsv = cv2.cvtColor(frame_org, cv2.COLOR_BGR2HSV)
# frame_gray = cv2.cvtColor(frame_org, cv2.COLOR_BGR2GRAY)
# mask_green = cv2.inRange(frame_hsv, l_green, u_green)
# mask_blue = cv2.inRange(frame_hsv, l_blue, u_blue)
# mask_blue[:40, :] = 0
# mask_black_org = cv2.inRange(frame_gray, 0, u_black) - mask_green - mask_blue

#~ Competition values
l_green = np.array([60, 80, 100], np.uint8) #alternate values: 50,50,90
u_green = np.array([80, 255, 255], np.uint8)
l_blue = np.array([100, 135, 40], np.uint8)
u_blue = np.array([115, 255, 255], np.uint8)

l_red = np.array([0, 130, 105], np.uint8)
u_red = np.array([10, 255, 255], np.uint8)
l_red = np.array([170, 96, 84], np.uint8) 
u_red = np.array([180, 255, 255], np.uint8)
crop_h_evactolt = 180
dp = 3
min_dist = 77 #67
param1 = 191 #128
param2 = 103 #62
min_radius = 65
max_radius = 88
u_blackforball = 49
u_black = 60
u_black_lineforltfromevac = 55 #! didnt tune (from 50)

frame_hsv = cv2.cvtColor(frame_org, cv2.COLOR_BGR2HSV)
frame_gray = cv2.cvtColor(frame_org, cv2.COLOR_BGR2GRAY)
mask_green = cv2.inRange(frame_hsv, l_green, u_green)
mask_blue = cv2.inRange(frame_hsv, l_blue, u_blue)
mask_blue[:40, :] = 0
mask_black_org = cv2.inRange(frame_gray, 0, u_black)



In [12]:

#^ Parameters that can consider tuning? TODO: analyse the effects of changing the various params
#^ 1. gap_check_h : essentially the region of interest for checking if there's a linegap (larger value results in faster/easier detection of linegap)
#^ 2. powered_y : cap on value hopefully cuts down on the wiggling
#^ 3. black_start_y and white_start_y values for endLineGap: lower black_start_y boundary means lines that are further away will also count as endlinegap;
#^    larger white_start_y boundary means shorter line segments/segments closer can be detected??


#^ other non essential stuff (for now):
#^ 1. x and y com (powering and shit)


#^ process elucidated:
#^ if no linegap, bot uses normal mask (w horizon crop)
#^ when bot detects linegap, switches to linegap_mask (triangle)
#^ bot then sweeps around using the mask, & tries to catch nearest segment of line (offsetted)

#^ some ideas to solve that random 90s?/wiggling
#^ pico side: check if 5s has passed and motors have barely moved 
#^ why does it not see that nook at the 90s hsdjfhjdsh
#^ reduce the threshold for endlinegap??? possibly by increasing white_start_y boundary???
#^ also pico side: js reverse back to original pos

#~constants
horizon_crop_h = 40 
gap_check_h = 155 #was 180

#~black masks
black_kernel = np.ones((7, 7), np.uint8)
black_linegap_kernel = np.ones((11,11), np.uint8)
mask_black = mask_black_org.copy()
mask_black = cv2.erode(mask_black, black_kernel)
mask_black = cv2.dilate(mask_black, black_kernel)

mask_uncropped_black = mask_black.copy()
mask_supercrop_black = mask_black.copy()
# mask_linegap = mask_black.copy()

mask_black[:horizon_crop_h, :] = 0
mask_supercrop_black[:-gap_check_h-25, :] = 0
mask_supercrop_black[-25:, :] = 0


In [13]:

#~ Finding the lowest black and white pixels in UNCROPPED black
black_row = np.amax(mask_uncropped_black, axis=1)
black_indices_v = np.where(black_row == 255)
black_start_y = black_indices_v[0][-1] if len(black_indices_v[0]) else 0
black_row[black_start_y:] = 255
white_indices = np.where(black_row == 0)
white_start_y = white_indices[0][-1] if len(white_indices[0]) else 0

black_line_height = black_start_y-white_start_y
print("black: ", black_start_y, "white:", white_start_y) #& debug height of line


#~ Finding next line segment
black_row[white_start_y:] = 0
black_indices_v2 = np.where(black_row == 255)
black_second_start_y = black_indices_v2[0][-1] if len(black_indices_v2[0]) else 0
black_row[black_second_start_y:] = 255
white_indices_v2 = np.where(black_row == 0)
white_second_start_y = white_indices_v2[0][-1] if len(white_indices_v2[0]) else 0
print("black: ", black_second_start_y, "white:", white_second_start_y) #& debug height of second line

#^ alternatively just do mask_black[white_start_y:, :] = 0 --> sets everything below white_start_y to black


#~ Finding left and right index of black
black_col = np.amax(mask_supercrop_black, axis=0)
black_indices_h = np.where(black_col == 255)
black_start_x = black_indices_h[0][0] if len(black_indices_h[0]) else 0
black_end_x = black_indices_h[0][-1] if len(black_indices_h[0]) else 0

black_line_width = black_end_x-black_start_x
print("black_line_width:", black_line_width) #& debug width of line


#~ Next line segment 
black_second_mask = mask_black.copy()
black_second_mask[:white_second_start_y, :] = 0
black_second_mask[black_second_start_y:, :] = 0
black_second_sum = np.sum(black_second_mask) /255
cv2.imshow("second line", black_second_mask)
print("black second pixels sum:", black_second_sum)

if black_second_sum > 0: #^ getting (offsetted) width of second line segment 
    black_second_row = np.amax(black_second_mask[black_second_start_y-35:black_second_start_y, :], axis=0)
    # black_second_row = np.amax(black_second_mask, axis=0) #? no difference????
    black_second_indices = np.where(black_second_row == 255)
    black_second_mask_start_x = black_second_indices[0][0] if len(black_second_indices[0]) else 0
    black_second_mask_end_x = black_second_indices[0][-1] if len(black_second_indices[0]) else 0
    black_second_mask_width = black_second_mask_end_x - black_second_mask_start_x

    print("second x width:", black_second_mask_width) #& debug width of line

else:
    black_second_mask_width = 0

cv2.waitKey(0)
cv2.destroyAllWindows()


black:  479 white: 346
black:  142 white: 95
black_line_width: 255
black second pixels sum: 3673.0
second x width: 89


In [30]:

#TODO: Find contours of the line or something idk
mask_xcontinuous = mask_black.copy()
mask_xcontinuous[:white_start_y, :] = 0


# contours, _ = cv2.findContours(mask_xcontinuous, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(frame_org, contours, -1, (0, 255, 0), 3)

# for i in contours:
#     if cv2.isContourConvex(i):
#         print(True)
        # cv2.drawContours(frame_org, i, -1, (0, 255, 0), 3)

# cv2.imshow("black continuous line", mask_xcontinuous)
cv2.imshow("contours mask", frame_org)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:

#& debug area
mask_linegap = mask_black.copy()
# mask_continuous = mask_continuous[white_second_start_y:black_second_start_y, black_start_x:black_end_x]

mask_linegap = cv2.bitwise_and(mask_gap, mask_linegap)

#~ If line gap (line ending and line width small) 
if ((black_start_y < height_lt-gap_check_h or white_start_y > height_lt-gap_check_h) and black_line_width < 300): #or (black_second_start_y > 150 and white_second_start_y < 150):
        if black_second_start_y <= 120 and not 80 <= black_second_mask_width <= 600: #or black_start_y < 200:
            print(">" * 15 + 'LINE GAP' + '<' * 15)
        mask_linegap_black = mask_black_org.copy()
        mask_linegap_black = cv2.erode(mask_linegap_black, kernel=black_linegap_kernel)
        mask_linegap_black = cv2.dilate(mask_linegap_black, kernel=black_linegap_kernel)
        mask_black = cv2.bitwise_and(mask_gap, mask_linegap_black)
        powered_y = 1

else:
    mask_black[:white_start_y, :] = 0

mask_linegap_black = mask_black_org.copy()
mask_linegap_black = cv2.erode(mask_linegap_black, kernel=black_linegap_kernel)
mask_linegap_black = cv2.dilate(mask_linegap_black, kernel=black_linegap_kernel)
mask_black = cv2.bitwise_and(mask_gap, mask_linegap_black)
cv2.imshow('black', mask_black)

# if black_start_y > 200 and white_start_y < 280 and black_line_width > 60: #TODO: offset mask (prev values are 400 and 250) 
#     print("-" * 40 + "end line gap" + "-" * 40)
#     end_line_gap = 1

if (black_start_y > 200 and white_start_y < 165 and black_line_width > 100) or (black_second_start_y > 120 and white_second_start_y < 110 and black_second_sum > 3500 and 70 < black_second_mask_width < 600): #!!! white2ndstart = 120, 80<black_second_mask_width<600
    # cv2.imshow("offset black mask", mask_black[200:250, :])
    print("-"*40 + "end line gap" + "-" * 40)
    
# cv2.imshow("black mask", mask_black)
# cv2.imshow("black uncropped mask", mask_uncropped_black)
cv2.imshow("super cropped mask", mask_supercrop_black)
# cv2.imshow("black linegap mask", mask_linegap)
cv2.waitKey(0)
cv2.destroyAllWindows()

----------------------------------------end line gap----------------------------------------


In [ ]:
x_com = np.tile(np.linspace(-1., 1., width), (height, 1))
y_com = np.array([[i] * width for i in np.linspace(1., 0, height)])
# print(y_com)

#uncomment if necessary
# white_start_y = 100
# mask_black[:white_start_y, :] = 0
# mask_black = cv2.bitwise_and(mask_gap, mask_black)
# powered_y = 2
powered_y = (height-40)/black_line_height if black_line_height != 0 else 1
powered_y = powered_y ** 0.5
print("y power: ", powered_y)
powered_y = min(3.5, powered_y)


# y_com = y_com ** powered_y
# x_com[:, :int(width/2)] *= -1
# x_com = x_com ** 1
# x_com[:, :int(width/2)] *= -1
y_black = cv2.bitwise_and(y_com, y_com, mask = mask_black)
x_black = cv2.bitwise_and(x_com, x_com, mask = mask_black)

cv2.imshow("yframe", y_black)
cv2.imshow("xframe", x_black)

y_resultant = np.mean(y_black) ** powered_y
x_resultant = np.mean(x_black)
print(y_resultant)
print(x_resultant)
angle = math.atan2(x_resultant, y_resultant) * 180/math.pi if y_resultant != 0 else 0
print(angle)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
x_com = np.tile(np.linspace(-1., 1., width), (height, 1)) #reps is (outside, inside)
# print(x_com)
# y_com = np.array([[i] * width for i in np.linspace(1., 1/height, height)]) #1/height is just to save pixels
# x_com_scale = ((1-y_com) ** 0.6)
y_com = np.array([[i] * width for i in np.linspace(1., 0, height)])
x_com_scale = ((1-y_com) ** 0.3)
x_com *= x_com_scale
print(x_com)

In [1]:
#* IMAGE PROCESSING THRESHOLDS FOR LINETRACK
frame_org = cv2.imread("hsvcalibimagegreeen.jpg")
u_black = 110 #^ DOM: prev value: 102 and 80

# ~ Real values
l_greenlt = np.array([60, 115, 100], np.uint8) #alternate values: 50,50,90
u_greenlt = np.array([80, 255, 255], np.uint8)

l_greenlt_alt = np.array([65, 55, 65], np.uint8)
u_greenlt_alt = np.array([80, 220, 255], np.uint8)
#~ My house's values
# l_green = np.array([70, 90, 80], np.uint8)
# u_green = np.array([96, 255, 255], np.uint8)

#~ Real values
l_blue = np.array([96, 170, 80], np.uint8)
u_blue = np.array([106, 245, 191], np.uint8)
#~ My house's values
# l_blue = np.array([96, 170, 80], np.uint8)
# u_blue = np.array([109, 245, 191], np.uint8)

# l_blue = np.array([96, 170, 80], np.uint8)
# u_blue = np.array([106, 245, 191], np.uint8)
# l_green = np.array([30, 50, 60], np.uint8)
# u_green = np.array([85, 255, 255], np.uint8)
# l_blue = np.array([96, 170, 80], np.uint8)
# u_blue = np.array([109, 245, 191], np.uint8)



frame_hsv = cv2.cvtColor(frame_org, cv2.COLOR_BGR2HSV)
frame_gray = cv2.cvtColor(frame_org, cv2.COLOR_BGR2GRAY)
mask_green = cv2.inRange(frame_hsv, l_green, u_green)
mask_blue = cv2.inRange(frame_hsv, l_blue, u_blue)
mask_blue[:40, :] = 0
mask_black_org = cv2.inRange(frame_gray, 0, u_black) - mask_blue

# mask_green = cv2.inRange(frame_hsv, l_greenlt, u_greenlt)
mask_green = cv2.inRange(frame_hsv, l_green, u_green)

mask_green_alt = cv2.inRange(frame_hsv, l_greenlt_alt, u_greenlt_alt)
mask_both_green = cv2.bitwise_and(mask_green, mask_green_alt)
mask_black_org = cv2.inRange(frame_gray, 0, u_black) - mask_both_green
# cv2.imshow('black frame', black_mask) #& debug black mask

mask_blue = cv2.inRange(frame_hsv, l_blue, u_blue) 
mask_blue[:horizon_crop_h, :] = 0
blue_sum = np.sum(mask_blue) / 255

mask_black = mask_black_org.copy()
mask_black = cv2.erode(mask_black, black_kernel)
mask_black = cv2.dilate(mask_black, black_kernel)
mask_uncropped_black = mask_black.copy()
mask_supercrop_black = mask_black.copy()

mask_black[:horizon_crop_h, :] = 0
# mask_supercrop_black[:-gap_check_h-25, :] = 0 #? shifted it up by 25 instead of 40
# mask_supercrop_black[-25:, :] = 0 
# end_line_gap = 0

# #~ Finding the lowest black and white pixels in UNCROPPED black
# black_row = np.amax(mask_uncropped_black, axis=1)
# black_indices_v = np.where(black_row == 255)
# black_start_y = black_indices_v[0][-1] if len(black_indices_v[0]) else 0
# black_row[black_start_y:] = 255
# white_indices = np.where(black_row == 0)
# white_start_y = white_indices[0][-1] if len(white_indices[0]) else 0

# black_line_height = black_start_y-white_start_y
# print("black start: ", black_start_y, "white: start", white_start_y)

# #~ Finding next line segment
# black_row[white_start_y:] = 0
# black_indices_v2 = np.where(black_row == 255)
# black_second_start_y = black_indices_v2[0][-1] if len(black_indices_v2[0]) else 0
# black_row[black_second_start_y:] = 255
# white_indices_v2 = np.where(black_row == 0)
# white_second_start_y = white_indices_v2[0][-1] if len(white_indices_v2[0]) else 0
# print("2nd black: ", black_second_start_y, "2nd white:", white_second_start_y) #& debug height of second line

# #~ Finding left and right index of black
# black_col = np.amax(mask_supercrop_black, axis=0)
# black_indices_h = np.where(black_col == 255)
# black_start_x = black_indices_h[0][0] if len(black_indices_h[0]) else 0
# black_end_x = black_indices_h[0][-1] if len(black_indices_h[0]) else 0
# black_line_width = black_end_x-black_start_x

# print("x width:", black_line_width) #& debug width of line

# #~ Next line segment 
# black_second_mask = mask_black.copy()
# black_second_mask[:white_second_start_y, :] = 0
# black_second_mask[black_second_start_y:, :] = 0
# black_second_sum = np.sum(black_second_mask) /255
# # cv2.imshow("second line", black_second_mask)
# print("black second pixels sum:", black_second_sum)

height_lt = 480
bot_stream_width = 640
#~ Line continuation
# else:

#~ Contours for weird close national tile
# if white_start_y < 140:
cv2.imshow("before contours", mask_black)
contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(frame_org, contours, -1, (0, 255, 0), 3)
cv2.imshow("with contours", frame_org)

cnts = []
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    cnts.append({
        "x": x,
        "y": y,
        "w": w,
        "h": h
    })
closest_contour = max(cnts, key=lambda x: x["y"]+x["h"])
# white_start_y = int(closest_contour["y"] - closest_contour["h"])
contour_mask = np.zeros([height_lt, bot_stream_width], dtype="uint8")
cv2.rectangle(contour_mask,(closest_contour["x"],closest_contour["y"]),(closest_contour["x"]+closest_contour["w"],closest_contour["y"]+closest_contour["h"]), 255 , -1)
# cv2.imshow("contours", contour_mask)

mask_black = cv2.bitwise_and(mask_black, contour_mask)

    # else:
    #     mask_black[:white_start_y, :] = 0
cv2.imshow("continuous", mask_black)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [22]:
l_red1evac = np.array([0, 150, 70], np.uint8)
u_red1evac = np.array([15, 255, 255], np.uint8)
l_red2evac = np.array([170, 150, 70], np.uint8) 
u_red2evac = np.array([180, 255, 255], np.uint8)

# l_red1evac = np.array([0, 160, 70], np.uint8)
# u_red1evac = np.array([10, 255, 255], np.uint8)
# l_red2evac = np.array([170, 160, 70], np.uint8) 
# u_red2evac = np.array([180, 255, 255], np.uint8)

frame_hsv = cv2.cvtColor(frame_org, cv2.COLOR_BGR2HSV)

mask_red1 = cv2.inRange(frame_hsv, l_red1evac, u_red1evac)
mask_red2 = cv2.inRange(frame_hsv, l_red2evac, u_red2evac)
mask_red = mask_red1 + mask_red2

redsum = np.sum(mask_red)/255
cv2.imshow("mask red", mask_red)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(redsum)

8672.0
